<a href="https://colab.research.google.com/github/ljbcoder/Project-Tensorflow/blob/main/PolicyIteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

grid_size = 4
states = grid_size **2
actions = ["up","down","left", "right"]
gamma = 1.0
theta = 1e-4

#Conditions for the grid world scenario
def step(state, action):
    row, col = divmod(state, grid_size) #Returns the quotient, remainder of state / grid_size
    if state in [0,15]:
      return state,0

    if action == "up":
      row = max(row-1,0)
    if action == "down":
      row = min(row+1,grid_size-1)
    if action == "left":
      col = max(col-1,0)
    if action == "right":
      col = min(col+1,grid_size-1)

    next_state = row * grid_size + col
    reward = -1
    return next_state, reward

#All possible probabilities for taking an action (For each state)
policy = {s: {a:0.25 for a in actions} for s in range(states)}  #Uniform random policy

#Initially start off with all zeros
V = np.zeros(states)


#Iterative Policy Evaluation
iteration = 0

while True:
  delta = 0
  new_V = np.copy(V)

  for s in range(states):
    #Calculate Value Function (@ State s)
    # Value function = sum(policy * sum(probability* (reward +  * gamma * value_function)))

    v= 0
    for a in actions:
      next_state, reward = step(s,a)
      v += policy[s][a] * (reward + gamma * V[next_state]) #We got rid of probability of going to next action because all 1

    #Alternative method to calculate V[s]:
    # V[s] = sum(policy[s][a] * (step(s,a)[1] + gamma * V[step(s,a)[0]]) for a in actions)
    # delta = max(delta, abs(v-V[s]))

    #Save new values into new_V
    new_V[s] = v
    delta = max(delta, abs(v-V[s]))

  V = new_V
  iteration += 1

  if delta < theta:
    break


print(f"Policy Evaluation took {iteration} iterations")
print(V.reshape(grid_size,grid_size))






Policy Evaluation took 173 iterations
[[  0.         -13.99893866 -19.99842728 -21.99824003]
 [-13.99893866 -17.99861452 -19.9984378  -19.99842728]
 [-19.99842728 -19.9984378  -17.99861452 -13.99893866]
 [-21.99824003 -19.99842728 -13.99893866   0.        ]]


In [ ]:
import numpy as np

grid_size = 4
states = grid_size **2
actions = ["up","down","left", "right"]
gamma = 1.0  #Discount Factor
theta = 1e-4  #Error Bound

#Conditions for the grid world scenario
def step(state, action):
    row, col = divmod(state, grid_size) #Returns the quotient, remainder of state / grid_size
    if state in [0,15]:
      return state,0

    if action == "up":
      row = max(row-1,0)
    if action == "down":
      row = min(row+1,grid_size-1)
    if action == "left":
      col = max(col-1,0)
    if action == "right":
      col = min(col+1,grid_size-1)

    next_state = row * grid_size + col
    reward = -1
    return next_state, reward

#All possible probabilities for taking an action (For each state)
policy = {s: {a:0.25 for a in actions} for s in range(states)}  #Uniform random policy

#Initially start off with all zeros
V = np.zeros(states)



iteration = 0
policy_stable = False

while not policy_stable:

  #Iterative Policy Evaluation
  while True:
    delta = 0
    new_V = np.copy(V)

    for s in range(states):
      #Calculate Value Function (@ State s)
      # Value function = sum(policy * sum(probability* (reward +  * gamma * value_function)))

      v= 0
      for a in actions:
        next_state, reward = step(s,a)
        v += policy[s][a] * (reward + gamma * V[next_state]) #We got rid of probability of going to next action because all 1

      #Alternative method to calculate V[s]:
      # V[s] = sum(policy[s][a] * (step(s,a)[1] + gamma * V[step(s,a)[0]]) for a in actions)
      # delta = max(delta, abs(v-V[s]))

      #Save new values into new_V
      new_V[s] = v
      delta = max(delta, abs(v-V[s]))

    V = new_V

    if delta < theta:
      break

  print(f"Iteration {iteration}: Value Function")
  print(V.reshape(grid_size,grid_size))
  print()

  print("Current Policy:")
  for s in range(states):
      if s in [0, 15]:
          print(f"State {s}: Terminal")
      else:
          # best_action = max(policy[s], key=policy[s].get)
          # print(f"State {s}: {best_action}")
          print(f"State {s}: {policy[s]}")

  for i in range(20):
    print("___",end="")
  print("\n")


  #Policy Improvement
  policy_stable = True
  for s in range(states):

    #Skip over states 0 and 15 (end-points)
    if s in [0,15]:
      continue

    old_action = max(policy[s], key=policy[s].get) #Get the highest probability action

    action_values = {}
    for a in actions:
      next_state, reward = step(s,a)
      action_values[a] = reward + gamma * V[next_state] #Compute q_pi (Action Value function)

    #Single best action to take:
    #best_action = max(action_values, key=action_values.get)

    #All best actions:
    best_value = max(action_values.values())
    best_actions = [a for a, val in action_values.items() if val == best_value]


    #Greedy policy: Probabilities for choosing action
    new_policy = {a: (1.0 / len(best_actions) if a in best_actions else 0.0) for a in actions}


    #Check if policy changed
    if new_policy != policy[s]:
      policy_stable = False

    #Note that new_policy is just the new policy for a given state s, we need to iterate through all states
    policy[s] = new_policy

  iteration += 1

print(f"Policy Iteration converged in {iteration} outer iterations.")
print("Final Value Function:")
print(V.reshape(grid_size, grid_size))

print("\nImproved Policy:")
for s in range(states):
    if s in [0, 15]:
        print(f"State {s}: Terminal")
    else:
        best_action = max(policy[s], key=policy[s].get)
        print(f"State {s}: {best_action}")











Iteration 0: Value Function
[[  0.         -13.99893866 -19.99842728 -21.99824003]
 [-13.99893866 -17.99861452 -19.9984378  -19.99842728]
 [-19.99842728 -19.9984378  -17.99861452 -13.99893866]
 [-21.99824003 -19.99842728 -13.99893866   0.        ]]

Current Policy:
State 0: Terminal
State 1: {'up': 0.25, 'down': 0.25, 'left': 0.25, 'right': 0.25}
State 2: {'up': 0.25, 'down': 0.25, 'left': 0.25, 'right': 0.25}
State 3: {'up': 0.25, 'down': 0.25, 'left': 0.25, 'right': 0.25}
State 4: {'up': 0.25, 'down': 0.25, 'left': 0.25, 'right': 0.25}
State 5: {'up': 0.25, 'down': 0.25, 'left': 0.25, 'right': 0.25}
State 6: {'up': 0.25, 'down': 0.25, 'left': 0.25, 'right': 0.25}
State 7: {'up': 0.25, 'down': 0.25, 'left': 0.25, 'right': 0.25}
State 8: {'up': 0.25, 'down': 0.25, 'left': 0.25, 'right': 0.25}
State 9: {'up': 0.25, 'down': 0.25, 'left': 0.25, 'right': 0.25}
State 10: {'up': 0.25, 'down': 0.25, 'left': 0.25, 'right': 0.25}
State 11: {'up': 0.25, 'down': 0.25, 'left': 0.25, 'right': 0.25}

In [ ]:
import numpy as np
from scipy.stats import poisson

MAX_CARS = 20
MAX_MOVE = 5
RENTAL_REWARD = 10
MOVE_COST = 2
GAMMA = 0.9
THRESHOLD = 1e-2

# Poisson parameters
RENTAL_REQUESTS = [3, 4]
RETURNS = [3, 2]

# Precompute Poisson probabilities up to 11
poisson_cache = dict()

def poisson_prob(n, lam):
    key = (n, lam)
    if key not in poisson_cache:
        poisson_cache[key] = poisson.pmf(n, lam)
    return poisson_cache[key]

# State: (cars at loc1, cars at loc2)
states = [(i, j) for i in range(MAX_CARS + 1) for j in range(MAX_CARS + 1)]
V = np.zeros((MAX_CARS + 1, MAX_CARS + 1))
policy = np.zeros((MAX_CARS + 1, MAX_CARS + 1), dtype=int)

# Expected return function
def expected_return(state, action, V):
    cars1, cars2 = state
    # Apply action
    action = int(action)
    cars1_ = min(cars1 - action, MAX_CARS)
    cars2_ = min(cars2 + action, MAX_CARS)
    if cars1_ < 0 or cars2_ < 0:
        return -np.inf  # Invalid action

    reward = -MOVE_COST * abs(action)
    expected = 0.0

    for rent1 in range(0, 11):
        prob_rent1 = poisson_prob(rent1, RENTAL_REQUESTS[0])
        real_rent1 = min(cars1_, rent1)
        ret_reward1 = real_rent1 * RENTAL_REWARD
        cars1_after_rent = cars1_ - real_rent1

        for rent2 in range(0, 11):
            prob_rent2 = poisson_prob(rent2, RENTAL_REQUESTS[1])
            real_rent2 = min(cars2_, rent2)
            ret_reward2 = real_rent2 * RENTAL_REWARD
            cars2_after_rent = cars2_ - real_rent2

            for ret1 in range(0, 11):
                prob_ret1 = poisson_prob(ret1, RETURNS[0])
                cars1_final = min(cars1_after_rent + ret1, MAX_CARS)

                for ret2 in range(0, 11):
                    prob_ret2 = poisson_prob(ret2, RETURNS[1])
                    cars2_final = min(cars2_after_rent + ret2, MAX_CARS)

                    prob = prob_rent1 * prob_rent2 * prob_ret1 * prob_ret2
                    total_reward = ret_reward1 + ret_reward2 + reward
                    expected += prob * (total_reward + GAMMA * V[cars1_final, cars2_final])
    return expected

# Policy Iteration
iteration = 0
while True:
    # Policy Evaluation
    while True:
        delta = 0
        for i in range(MAX_CARS + 1):
            for j in range(MAX_CARS + 1):
                v = V[i, j]
                action = policy[i, j]
                V[i, j] = expected_return((i, j), action, V)
                delta = max(delta, abs(v - V[i, j]))
        if delta < THRESHOLD:
            break

    # Policy Improvement
    policy_stable = True
    for i in range(MAX_CARS + 1):
        for j in range(MAX_CARS + 1):
            old_action = policy[i, j]
            action_returns = []
            for a in range(-MAX_MOVE, MAX_MOVE + 1):
                if 0 <= i - a <= MAX_CARS and 0 <= j + a <= MAX_CARS:
                    val = expected_return((i, j), a, V)
                    action_returns.append((val, a))
            _, best_action = max(action_returns)
            policy[i, j] = best_action
            if old_action != best_action:
                policy_stable = False

    iteration += 1
    print(f"Iteration {iteration} complete.")
    if policy_stable:
        print("Policy stable — converged.")
        break


# Print results
print("\nFinal Policy (action = cars moved from loc1 to loc2):")
print(policy)
print("\nFinal Value Function:")
print(np.round(V, 1))
